# PYNQ tutorial: DMA to streamed interfaces

Overlay consists of two DMAs and an AXI Stream FIFO (input and output AXI stream interfaces). The FIFO represents an accelerator.

* The first DMA with read channel enabled is connected from DDR to IP input stream (reading from DDR, and sending to AXI stream).
* The second DMA has a write channel enabled and is connected to IP output stream to DDR (receiving from AXI stream, and writing to DDR memory).

There are other IP in the design which will be ignored for now.

![](data/dma_stream_example.png)

## 1. Downloading overlay
The overlay can be downloaded automatically when instantiating an overlay class.

In [1]:
from pynq.overlays.pynqtutorial import PynqTutorialOverlay

overlay = PynqTutorialOverlay("pynqtutorial.bit")

We can also check all the IPs on this overlay.

In [2]:
from pprint import pprint

pprint(overlay.ip_dict)

{'axi_dma_from_pl_to_ps': {'addr_range': 65536,
                           'driver': <class 'pynq.lib.dma.DMA'>,
                           'fullpath': 'axi_dma_from_pl_to_ps',
                           'gpio': {},
                           'interrupts': {'s2mm_introut': {'controller': 'system_interrupts',
                                                           'fullpath': 'axi_dma_from_pl_to_ps/s2mm_introut',
                                                           'index': 1}},
                           'phys_addr': 1078001664,
                           'state': None,
                           'type': 'xilinx.com:ip:axi_dma:7.1'},
 'axi_dma_from_ps_to_pl': {'addr_range': 65536,
                           'driver': <class 'pynq.lib.dma.DMA'>,
                           'fullpath': 'axi_dma_from_ps_to_pl',
                           'gpio': {},
                           'interrupts': {'mm2s_introut': {'controller': 'system_interrupts',
                                       

## 2. Create DMA instances

Using the IP dictionary listed above, we can just provide the corresponding dictionary as an input parameter to the DMA python class.

In [3]:
from pynq.lib import DMA

dma_ps2pl_description = overlay.ip_dict['axi_dma_from_ps_to_pl']
dma_ps2pl = DMA(dma_ps2pl_description)

dma_pl2ps_description = overlay.ip_dict['axi_dma_from_pl_to_ps']
dma_pl2ps = DMA(dma_pl2ps_description)

## 3. Debug DMA

In the following cell, we will create some useful functions to print control and status information from the DMA instances.

In [4]:
from pynq import MMIO

overlay.ip_dict['axi_dma_from_ps_to_pl']['phys_addr']


dma_ps2pl_controller = MMIO(
    dma_ps2pl_description['phys_addr'], 128)
dma_pl2ps_controller = MMIO(
    dma_pl2ps_description['phys_addr'], 128)

def print_dma_status():
    print("====  From memory to FIFO  ====")
    print("MM to Stream Control: 0x" + 
          format(dma_ps2pl_controller.read(0x0), '02x'))
    print("             Binary : 0b" + 
          format(dma_ps2pl_controller.read(0x0), '0b'))
    print("MM to Stream Status : 0x" + 
          format(dma_ps2pl_controller.read(0x4), '02x'))
    print("             Binary : 0b" + 
          format(dma_ps2pl_controller.read(0x4), '0b'))
    
    print("\n==== From FIFO to Memory ====")
    print("Stream to MM Control: 0x" + 
          format(dma_pl2ps_controller.read(0x30), '02x'))
    print("             Binary : 0b" + 
          format(dma_pl2ps_controller.read(0x30), '0b'))
    print("Stream to MM Status : 0x" + 
          format(dma_pl2ps_controller.read(0x34), '02x'))
    print("             Binary : 0b" + 
          format(dma_pl2ps_controller.read(0x34), '0b'))

The bits for control registers are:
* bit 1 : This bit is always 1.
* bit 16 : IRQ threshold.

The bits for status registers are:
* bit 0 : If it is 1, it means the DMA is halted.

In [5]:
print_dma_status()

====  From memory to FIFO  ====
MM to Stream Control: 0x11003
             Binary : 0b10001000000000011
MM to Stream Status : 0x00
             Binary : 0b0

==== From FIFO to Memory ====
Stream to MM Control: 0x11003
             Binary : 0b10001000000000011
Stream to MM Status : 0x00
             Binary : 0b0


## 4. Read DMA
We will read some data from memory, and write to FIFO in the following cells.

The first step is to create the a contiguous memory block. We will use the `Xlnk()` driver.

In [6]:
import numpy as np
from pynq import Xlnk

data_size = 10
mmu = Xlnk()

The CMA array created by this driver can be used similarly as a numpy array. We can write some test data to the array. The data will be transferred by the DMA to the FIFO. 

In [7]:
cma_array_send = mmu.cma_array([1, data_size])
for i in range(data_size):
    cma_array_send[0][i] = i + 0xcafe0000

Let's check the contents of the array. The data in the following cell will be sent from PS (DDR memory) to PL (streaming FIFO).

In [8]:
for i in range(data_size):
    print('0x' + format(cma_array_send[0][i], '02x'))

0xcafe0000
0xcafe0001
0xcafe0002
0xcafe0003
0xcafe0004
0xcafe0005
0xcafe0006
0xcafe0007
0xcafe0008
0xcafe0009


Now we are ready to carry out DMA transfer from a memory block in DDR to FIFO.

In [9]:
dma_ps2pl.sendchannel.transfer(cma_array_send)

Let's check the status of the DMA.

Control register:
* bit 0 : Start the DMA if the value is 1, else halt the DMA.
* bit 1 : Always 1
* bit 16 : IRQ threshold.

Status register:
* bit 0 : 0 means the DMA is halted.
* bit 1 : 1 means the DMA is idle.
* bit 12 : Interrupt on complete.

In [10]:
print_dma_status()

====  From memory to FIFO  ====
MM to Stream Control: 0x11003
             Binary : 0b10001000000000011
MM to Stream Status : 0x1002
             Binary : 0b1000000000010

==== From FIFO to Memory ====
Stream to MM Control: 0x11003
             Binary : 0b10001000000000011
Stream to MM Status : 0x00
             Binary : 0b0


## 5. Write DMA
Let's read the data back from FIFO stream, and write to MM memory. The steps are similar.

We will prepare an empty array before reading data back from FIFO.

In [11]:
cma_array_recv = mmu.cma_array([1, data_size])
for i in range(data_size):
    print('0x' + format(cma_array_recv[0][i], '02x'))

0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00


In [12]:
dma_pl2ps.recvchannel.transfer(cma_array_recv)

The next cell will print out the data received from PL (streaming FIFO) to PS (DDR memory). This should be the same as the data we sent previously.

In [13]:
for i in range(data_size):
    print('0x' + format(cma_array_recv[0][i], '02x'))

0xcafe0000
0xcafe0001
0xcafe0002
0xcafe0003
0xcafe0004
0xcafe0005
0xcafe0006
0xcafe0007
0xcafe0008
0xcafe0009


In [14]:
print_dma_status()

====  From memory to FIFO  ====
MM to Stream Control: 0x11003
             Binary : 0b10001000000000011
MM to Stream Status : 0x1002
             Binary : 0b1000000000010

==== From FIFO to Memory ====
Stream to MM Control: 0x11003
             Binary : 0b10001000000000011
Stream to MM Status : 0x1002
             Binary : 0b1000000000010


## 6. Free all the memory buffers
Don't forget to free all the memory buffers to avoid memory leaks!

In [15]:
mmu.xlnk_reset()